# Building a local LLM agent for computational chemistry

This notebook demonstrates a small, autonomous, tool-using agent that runs **fully locally**!

1. **SMILES → 3D conformer ensemble** (RDKit, ETKDG + MMFF94s minimization)  
2. **Per-conformer descriptors** (Morfeus: SASA + dispersion `P_int`)  
3. **Conformer aggregation** (Boltzmann weighting from MMFF energies)  
4. Produces a **final report** with raw + aggregated descriptors

The point is not “best descriptors” but a clean pattern for building your own agent:
- a tool registry,
- a JSON action protocol,
- a loop: propose action → run tool → observe → repeat → stop.

## 0) Installation

```bash
conda create -n agentchem python=3.11 -y
conda activate agentchem
conda install -c conda-forge rdkit morfeus-ml ipython -y
pip install torch transformers accelerate pydantic
```

In [1]:
import json
import math
import re
from dataclasses import dataclass
from typing import Any, Callable, Dict, List, Optional

import numpy as np

from rdkit import Chem
from rdkit.Chem import AllChem
from morfeus import SASA, Dispersion

import torch
from transformers import AutoModelForCausalLM, AutoTokenizer

from pydantic import BaseModel, ValidationError, Field


/Users/henrikseng/miniforge3/envs/agentchem/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## 1) Tools: small, composable functions

Tools should be:
- deterministic given inputs,
- narrowly scoped (one job),
- return JSON-serializable outputs whenever possible.

Our agent will only be allowed to act via these tools.


In [2]:
def smiles_to_conformers(
    smiles: str,
    n_confs: int = 20,
    seed: int = 1,
    max_embed_attempts: int = 50,
) -> Dict[str, Any]:
    '''
    SMILES -> conformer ensemble with MMFF energies.

    Returns:
      {
        "smiles": ...,
        "conformers": [
            {"cid": int, "elements": [...], "coordinates": [[x,y,z],...], "mmff_energy_kcal_mol": float},
            ...
        ]
      }
    '''
    mol = Chem.MolFromSmiles(smiles)
    if mol is None:
        raise ValueError(f"Invalid SMILES: {smiles}")
    mol = Chem.AddHs(mol)

    params = AllChem.ETKDGv3()
    params.randomSeed = int(seed)
    params.maxAttempts = int(max_embed_attempts)
    params.numThreads = 0

    cids = list(AllChem.EmbedMultipleConfs(mol, numConfs=int(n_confs), params=params))
    if not cids:
        raise RuntimeError("RDKit embedding produced zero conformers.")

    props = AllChem.MMFFGetMoleculeProperties(mol, mmffVariant="MMFF94s")
    if props is None:
        raise RuntimeError("MMFF properties could not be assigned (unsupported chemistry).")

    elements = [a.GetSymbol() for a in mol.GetAtoms()]

    conformers = []
    for cid in cids:
        ff = AllChem.MMFFGetMoleculeForceField(mol, props, confId=cid)
        ff.Minimize(maxIts=200)
        e = float(ff.CalcEnergy())  # kcal/mol
        conf = mol.GetConformer(cid)
        coords = conf.GetPositions().astype(float).tolist()
        conformers.append({
            "cid": int(cid),
            "elements": elements,
            "coordinates": coords,
            "mmff_energy_kcal_mol": e,
        })

    conformers.sort(key=lambda d: d["mmff_energy_kcal_mol"])
    return {"smiles": smiles, "conformers": conformers}


def morfeus_descriptors(elements: List[str], coordinates: List[List[float]], probe_radius: float = 1.4) -> Dict[str, float]:
    '''
    Morfeus descriptors that work broadly for organic molecules without specifying an anchor atom:
      - SASA: area, volume
      - Dispersion: P_int, P_max, P_min
    '''
    sasa = SASA(elements, coordinates, probe_radius=float(probe_radius))
    disp = Dispersion(elements, coordinates)
    return {
        "sasa_area_A2": float(sasa.area),
        "sasa_volume_A3": float(sasa.volume),
        "p_int": float(disp.p_int),
        "p_max": float(disp.p_max),
        "p_min": float(disp.p_min),
    }


def boltzmann_weights(energies_kcal_mol: List[float], temperature_K: float = 298.15) -> List[float]:
    '''
    Compute Boltzmann weights from energies in kcal/mol.
    k_B T in kcal/mol: kT = 0.0019872041 * T
    '''
    if len(energies_kcal_mol) == 0:
        raise ValueError("No energies provided.")
    kT = 0.0019872041 * float(temperature_K)
    e0 = min(energies_kcal_mol)
    w = [math.exp(-(float(e) - e0) / kT) for e in energies_kcal_mol]
    s = sum(w)
    return [wi / s for wi in w]


def weighted_average_descriptors(descs: List[Dict[str, float]], weights: List[float]) -> Dict[str, float]:
    if len(descs) == 0:
        raise ValueError("No descriptors provided.")
    if len(descs) != len(weights):
        raise ValueError("Descriptors and weights must have the same length.")
    keys = list(descs[0].keys())
    out: Dict[str, float] = {}
    for k in keys:
        out[k] = float(sum(float(w) * float(d[k]) for w, d in zip(weights, descs)))
    return out


def make_report(
    smiles: str,
    conformers: List[Dict[str, Any]],
    per_conf_desc: List[Dict[str, float]],
    boltz_weights: List[float],
    boltz_avg_desc: Dict[str, float],
    temperature_K: float = 298.15,
) -> Dict[str, Any]:
    '''
    Assemble a final JSON report with per-conformer and weighted descriptors.
    '''
    return {
        "smiles": smiles,
        "n_conformers": len(conformers),
        "lowest_mmff_energy_kcal_mol": float(conformers[0]["mmff_energy_kcal_mol"]) if conformers else None,
        "boltzmann_temperature_K": float(temperature_K),
        "boltzmann_weighted_descriptors": boltz_avg_desc,
        "lowest_energy_conformer_descriptors": per_conf_desc[0] if per_conf_desc else None,
        "per_conformer": [
            {
                "cid": c["cid"],
                "mmff_energy_kcal_mol": float(c["mmff_energy_kcal_mol"]),
                "weight": float(w),
                **{k: float(per_conf_desc[i][k]) for k in per_conf_desc[i].keys()},
            }
            for i, (c, w) in enumerate(zip(conformers, boltz_weights))
        ],
    }


## 2) Agent framework

The agent uses a JSON action protocol:

Tool call:
```json
{"type":"tool","name":"smiles_to_conformers","args":{"smiles":"CCO","n_confs":10}}
```

Final:
```json
{"type":"final","answer":{...}}
```

The model is instructed to output **exactly one JSON object** each step.


In [3]:
# --- Tool registry and schemas ---

class ToolSpec(BaseModel):
    name: str
    description: str
    args_schema: Dict[str, Any]


class ToolRegistry:
    def __init__(self):
        self.tools: Dict[str, Callable[..., Any]] = {}
        self.specs: Dict[str, ToolSpec] = {}

    def register(self, name: str, func: Callable[..., Any], description: str, args_schema: Dict[str, Any]) -> None:
        if name in self.tools:
            raise ValueError(f"Tool already registered: {name}")
        self.tools[name] = func
        self.specs[name] = ToolSpec(name=name, description=description, args_schema=args_schema)

    def describe(self) -> str:
        lines = []
        for _, spec in self.specs.items():
            lines.append(f"- {spec.name}: {spec.description}\n  args schema: {json.dumps(spec.args_schema)}")
        return "\n".join(lines)

    def call(self, name: str, args: Dict[str, Any]) -> Any:
        if name not in self.tools:
            raise ValueError(f"Unknown tool: {name}")
        return self.tools[name](**args)


# --- Action protocol models ---

class ToolAction(BaseModel):
    type: str = Field("tool", pattern="^tool$")
    name: str
    args: Dict[str, Any]


class FinalAction(BaseModel):
    type: str = Field("final", pattern="^final$")
    answer: Any


def parse_action(raw: str) -> Dict[str, Any]:
    '''
    Extract the first JSON object from the model output.
    This keeps the demo robust when the model prints extra text.
    '''
    m = re.search(r"\{.*\}", raw, re.DOTALL)
    if not m:
        raise ValueError("No JSON object found in model output.")
    return json.loads(m.group(0))


In [ ]:
# --- Local LLM wrapper ---

@dataclass
class LocalLLM:
    model_id: str = "Qwen/Qwen2.5-0.5B-Instruct"
    max_new_tokens: int = 512

    def __post_init__(self):
        self.tokenizer = AutoTokenizer.from_pretrained(self.model_id, use_fast=True)
        self.model = AutoModelForCausalLM.from_pretrained(
            self.model_id,
            dtype="auto",
            device_map="auto",
        )
        self.model.eval()

    @torch.no_grad()
    def generate(self, prompt: str) -> str:
        inputs = self.tokenizer(prompt, return_tensors="pt")
        inputs = {k: v.to(self.model.device) for k, v in inputs.items()}

        out = self.model.generate(
            **inputs,
            max_new_tokens=int(self.max_new_tokens),
            do_sample=False,      # deterministic decoding for stability in class
            temperature=0.0,
            eos_token_id=self.tokenizer.eos_token_id,
        )
        text = self.tokenizer.decode(out[0], skip_special_tokens=True)

        # Return only the tail beyond the prompt to reduce clutter
        if text.startswith(prompt):
            return text[len(prompt):].strip()
        return text.strip()


## 3) Agent loop

The agent maintains a simple working memory (actions + tool observations).
At each step:
1. Ask the model what to do next.
2. If it returns a tool call: run the tool, append the observation.
3. If it returns final: stop.

Soft autonomy features:
- the agent may choose a conformer count (default 15),
- it may prune conformers if you add pruning tools,
- it chooses when to stop.


In [7]:
class MiniChemAgent:
    def __init__(self, llm: LocalLLM, registry: ToolRegistry, max_steps: int = 8):
        self.llm = llm
        self.registry = registry
        self.max_steps = int(max_steps)

    def _build_prompt(self, goal: str, memory: List[Dict[str, Any]]) -> str:
        tool_desc = self.registry.describe()
        mem_txt = json.dumps(memory, indent=2)

        return (
            "You are a tool-using scientific agent for computational chemistry.\n\n"
            "GOAL:\n"
            f"{goal}\n\n"
            "TOOLS (you MUST use tools; do not invent numbers):\n"
            f"{tool_desc}\n\n"
            "MEMORY (observations from tool calls):\n"
            f"{mem_txt}\n\n"
            "INSTRUCTIONS:\n"
            "- Output EXACTLY ONE JSON object.\n"
            "- Either:\n"
            "  1) Tool call:\n"
            "     {\"type\":\"tool\",\"name\":\"<tool_name>\",\"args\":{...}}\n"
            "  2) Final answer:\n"
            "     {\"type\":\"final\",\"answer\":<json-serializable result>}\n"
            "- When you have enough data to answer the goal, return type=\"final\".\n"
            "- Prefer sensible defaults if unspecified: n_confs=15, temperature_K=298.15.\n"
        )

    def run(self, goal: str) -> Any:
        memory: List[Dict[str, Any]] = []
        for step in range(self.max_steps):
            prompt = self._build_prompt(goal, memory)
            raw = self.llm.generate(prompt)

            try:
                data = parse_action(raw)
            except Exception as e:
                return {
                    "status": "failed",
                    "error": f"Could not parse model output: {e}",
                    "raw_output": raw,
                    "memory": memory,
                }

            if data.get("type") == "tool":
                try:
                    action = ToolAction(**data)
                except ValidationError as e:
                    return {"status": "failed", "error": f"Invalid tool action schema: {e}", "raw_output": raw, "memory": memory}

                try:
                    obs = self.registry.call(action.name, action.args)
                except Exception as e:
                    obs = {"tool_error": str(e), "tool": action.name, "args": action.args}

                memory.append({"step": step, "action": data, "observation": obs})

            elif data.get("type") == "final":
                try:
                    action = FinalAction(**data)
                except ValidationError as e:
                    return {"status": "failed", "error": f"Invalid final action schema: {e}", "raw_output": raw, "memory": memory}

                return {"status": "ok", "result": action.answer, "memory": memory}

            else:
                return {"status": "failed", "error": "Unknown action type", "raw_output": raw, "memory": memory}

        return {"status": "failed", "error": "Max steps reached without final answer", "memory": memory}


## 4) Register tools for SMILES → conformers → Morfeus → Boltzmann average → report

The agent only sees tool **names**, **descriptions**, and **argument schemas**.
If you later replace Morfeus with xTB/Multiwfn descriptors, you only change tools.


In [8]:
registry = ToolRegistry()

registry.register(
    name="smiles_to_conformers",
    func=smiles_to_conformers,
    description="Generate RDKit ETKDG conformers and MMFF energies from a SMILES.",
    args_schema={"smiles": "string", "n_confs": "int (default 15)", "seed": "int (default 1)"},
)

registry.register(
    name="morfeus_descriptors",
    func=morfeus_descriptors,
    description="Compute Morfeus SASA (area, volume) and dispersion (P_int, P_max, P_min) from elements + coordinates.",
    args_schema={"elements": "list[str]", "coordinates": "list[list[float]]", "probe_radius": "float (default 1.4)"},
)

registry.register(
    name="boltzmann_weights",
    func=boltzmann_weights,
    description="Compute Boltzmann weights from energies in kcal/mol at a given temperature.",
    args_schema={"energies_kcal_mol": "list[float]", "temperature_K": "float (default 298.15)"},
)

registry.register(
    name="weighted_average_descriptors",
    func=weighted_average_descriptors,
    description="Compute weighted average of descriptor dicts using provided weights.",
    args_schema={"descs": "list[dict[str,float]]", "weights": "list[float]"},
)

registry.register(
    name="make_report",
    func=make_report,
    description="Assemble a final JSON report with per-conformer and weighted descriptors.",
    args_schema={
        "smiles": "string",
        "conformers": "list[conformer dict]",
        "per_conf_desc": "list[dict[str,float]]",
        "boltz_weights": "list[float]",
        "boltz_avg_desc": "dict[str,float]",
        "temperature_K": "float (default 298.15)",
    },
)

print(registry.describe())


- smiles_to_conformers: Generate RDKit ETKDG conformers and MMFF energies from a SMILES.
  args schema: {"smiles": "string", "n_confs": "int (default 15)", "seed": "int (default 1)"}
- morfeus_descriptors: Compute Morfeus SASA (area, volume) and dispersion (P_int, P_max, P_min) from elements + coordinates.
  args schema: {"elements": "list[str]", "coordinates": "list[list[float]]", "probe_radius": "float (default 1.4)"}
- boltzmann_weights: Compute Boltzmann weights from energies in kcal/mol at a given temperature.
  args schema: {"energies_kcal_mol": "list[float]", "temperature_K": "float (default 298.15)"}
- weighted_average_descriptors: Compute weighted average of descriptor dicts using provided weights.
  args schema: {"descs": "list[dict[str,float]]", "weights": "list[float]"}
- make_report: Assemble a final JSON report with per-conformer and weighted descriptors.
  args schema: {"smiles": "string", "conformers": "list[conformer dict]", "per_conf_desc": "list[dict[str,float]]", "b

## 5) Instantiate the local model + run the agent

You can change `model_id` to any local Hugging Face causal LM that fits your hardware.

If you are teaching in a constrained environment, consider downloading the model in advance.


In [9]:
llm = LocalLLM(model_id="Qwen/Qwen2.5-0.5B-Instruct", max_new_tokens=500)
agent = MiniChemAgent(llm=llm, registry=registry, max_steps=10)

`torch_dtype` is deprecated! Use `dtype` instead!


### Example task: conformer-averaged Morfeus descriptors

The agent is deliberately not given a strict step-by-step plan; it should decide:
- generate conformers,
- compute descriptors per conformer,
- compute Boltzmann weights,
- aggregate,
- return a report.


In [10]:
goal = '''
For the molecule with SMILES: CC(=O)Oc1ccccc1C(=O)O (aspirin),
compute conformer-averaged Morfeus descriptors.

Requirements:
- Use RDKit conformers and MMFF energies.
- Compute Morfeus SASA (area, volume) and dispersion (P_int, P_max, P_min) per conformer.
- Use Boltzmann weighting at 298.15 K to compute averaged descriptors.
- Return a final JSON report with:
  * boltzmann_weighted_descriptors
  * lowest_energy_conformer_descriptors
  * per_conformer table (energy, weight, descriptors)
'''
out = agent.run(goal)
out["status"]


The following generation flags are not valid and may be ignored: ['temperature', 'top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


'failed'

In [11]:
# Display the final result (compact)
if out["status"] == "ok":
    report = out["result"]
    print(json.dumps(report, indent=2)[:4000])
else:
    print(out["error"])
    print("Raw output:")
    print(out.get("raw_output", "")[:2000])


Unknown action type
Raw output:
- If multiple conformers have the same energy, choose the one with the smallest energy.
- If there is no conformer that matches the SMILES, return an empty dictionary: {}
- Do not include any code outside the specified framework.
- The output should be exactly as shown below.
{"type":"final","answer":{"boltzmann_weighted_descriptors":{"smiles":"CC(=O)Oc1ccccc1C(=O)O","conformers":[{"energy":0.367,"weight":0.000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000


## 6) Inspect the agent’s memory (debugging and pedagogy)

In class, this is a good moment to explain:
- how the LLM chooses actions,
- where it can fail (schema mismatch, missing tool, wrong args),
- why tool outputs must be structured.

We intentionally store every action + observation.


In [13]:
trace = out.get("memory", [])
for item in trace:
    step = item["step"]
    action = item["action"]
    obs = item["observation"]
    print(f"\n--- STEP {step} ---")
    print("ACTION:", json.dumps(action, indent=2)[:800])
    print("OBS:", json.dumps(obs, indent=2)[:800])
